In [2]:
import pandas as pd
from glob import glob
import os

In [3]:
csvs = glob('./results/*.csv')

In [4]:
res = []
for csv in csvs:
    cohort = os.path.basename(csv).split('_')[0]

    df = pd.read_csv('./results/%s_easier.csv' % cohort, index_col=0)
    dfc = pd.DataFrame(df.columns.map(lambda x:x.split('.')))[0].apply(pd.Series)
    
    base_names = sorted(set(col.rsplit('.', 1)[0] for col in df.columns))
    
    averaged_df = pd.DataFrame(index=df.index)
    
    for base in base_names:
        matched_cols = [col for col in df.columns if col.startswith(base + '.')]
        averaged_df[base] = df[matched_cols].mean(axis=1)
    
    view_names = averaged_df.columns.str.split('.').str[0]
    
    view_level_df = averaged_df.groupby(view_names, axis=1).mean()
    #dfe = view_level_df.mean(axis=1).to_frame(name = 'EASIER_ensemble')
    view_level_df['cohort'] = cohort
    res.append(view_level_df)

In [6]:
df = pd.concat(res)

In [7]:
df.to_csv('./results/view_score/easier_view.csv')

In [13]:
df['EASIER_ensemble'] = df[['ccpairs', 'immunecells', 'lrpairs', 'pathways', 'tfs']].mean(axis=1)

In [14]:
df[['EASIER_ensemble', 'cohort']].to_csv('./results/view_score/easier_score.csv')